In [1]:
using Revise
!(pwd() in LOAD_PATH) ? push!(LOAD_PATH, pwd()) : nothing
include("SISEPUEDEPJSF.jl")
import .SISEPUEDEPJSF



using Pkg
Pkg.activate(".")


using DataFrames
using NemoMod
using SQLite
using Cbc
using JuMP
using Gurobi
using GAMS

include("setup_analysis.jl")

  Activating project at `~/Documents/Projects/git_jbus/lac_decarbonization/julia`


"/Users/jsyme/Documents/Projects/git_jbus/lac_decarbonization/tmp/nemomod_intermediate_database.sqlite"

In [47]:
fp_sqlite = "/Users/jsyme/Documents/Projects/git_jbus/lac_decarbonization/tmp/nemomod_intermediate_database.sqlite"
db = SQLite.DB(fp_sqlite)


SQLite.DB("/Users/jsyme/Documents/Projects/git_jbus/lac_decarbonization/tmp/nemomod_intermediate_database.sqlite")

In [52]:
model = Model(Gurobi.Optimizer); #set_silent(model)
@time NemoMod.calculatescenario(
    fp_sqlite; 
    jumpmodel = model,
    numprocs = 1
)





2023-23-Feb 14:38:18.899 Started modeling scenario.
2023-23-Feb 14:38:18.900 Validated run-time arguments.
2023-23-Feb 14:38:18.903 Connected to scenario database. Path = /Users/jsyme/Documents/Projects/git_jbus/lac_decarbonization/tmp/nemomod_intermediate_database.sqlite.
2023-23-Feb 14:38:18.910 Dropped pre-existing result tables from database.
2023-23-Feb 14:38:18.910 Verified that transmission modeling is not enabled.
2023-23-Feb 14:38:19.017 Created parameter views and indices.
2023-23-Feb 14:38:19.022 Created temporary tables.
2023-23-Feb 14:38:19.755 Executed core database queries.
2023-23-Feb 14:38:19.757 Defined dimensions.
2023-23-Feb 14:38:19.776 Defined demand variables.
2023-23-Feb 14:38:19.790 Defined storage variables.
2023-23-Feb 14:38:19.793 Defined capacity variables.
2023-23-Feb 14:38:19.997 Defined activity variables.
2023-23-Feb 14:38:20.010 Defined costing variables.
2023-23-Feb 14:38:20.011 Defined reserve margin variables.
2023-23-Feb 14:38:20.066 Defined emissi

OPTIMAL::TerminationStatusCode = 1

In [50]:
dict_tables = Dict()

tabs = SQLite.tables(db)[1]
table_name = 0
tabs_populated = Vector{String}()
for k in 1:length(tabs)
    table_name = tabs[k]
    df_tmp = DBInterface.execute(db, "select * from $(table_name)") |> DataFrame
    dict_tables[table_name] = df_tmp
    
    (nrow(df_tmp) > 0) ? push!(tabs_populated, table_name) : nothing
end


In [64]:
get(dict_tables, "REGION", nothing)

,val,desc
,String,String
1,costa_rica,Costa Rica


In [63]:
for k in sort(collect(keys(dict_tables)))
    
    (string(k[1:1]) == "v") ? println(get(dict_tables, k, nothing)) : nothing
end

0×6 DataFrame
0×5 DataFrame
0×6 DataFrame
0×6 DataFrame
0×5 DataFrame
0×4 DataFrame
0×6 DataFrame
0×6 DataFrame


In [48]:
#df = get(dict_tables, "CapacityFactor", nothing)
#filter(x -> ((x[:y] == "1000") & (x[:t] == "pp_hydropower")), df)
#DBInterface.execute(db, "drop * from SpecifiedDemandProfile")
table_name = "SpecifiedDemandProfile"
DBInterface.execute(db, "delete from $(table_name)")

SQLite.Query(SQLite.Stmt(SQLite.DB("/Users/jsyme/Documents/Projects/git_jbus/lac_decarbonization/tmp/nemomod_intermediate_database.sqlite"), 1), Base.RefValue{Int32}(101), Symbol[], Type[], Dict{Symbol, Int64}(), Base.RefValue{Int64}(0))

In [36]:
#keys(dict_tables)
for tab in sort(collect(keys(dict_tables)))
    
    df = get(dict_tables, tab, nothing)
    #if "r" in names(df)
    #    println(tab)
    #    all_regions = Set(df[:, :r])
    #    println("regions:\t\n$(all_regions)")
    #    (nrow(df) == 0) ? println("**NO ROWS**\n\n") : println("\n\n")
    
    if "id" in names(df)
        s_id = Set(df[:, :id])
        (length(s_id) != nrow(df)) ? println("ID MISTMATCH IN $(tab)") : nothing
    end
    
end



In [18]:
get(dict_tables, "vtotalcapacityannual", nothing)

,r,t,y,val,solvedtm
,String?,String?,String?,Float64?,String?


In [126]:
filter(x -> (x[:f] != "fuel_electricity"), get(dict_tables, "SpecifiedAnnualDemand", nothing))

,id,r,f,y,val
,Int64,String,String,String,Float64
1,4,costa_rica,fuel_oil,1000,72.0
2,5,costa_rica,fuel_oil,1001,73.5
3,6,costa_rica,fuel_oil,1002,75.0


In [99]:
sort([x for x in keys(dict_tables) if String(x[1:1]) == "v"])


8-element Vector{String}:
 "vdemandnn"
 "vnewcapacity"
 "vproductionbytechnologyannual"
 "vproductionnn"
 "vtotalcapacityannual"
 "vtotaldiscountedcost"
 "vusebytechnologyannual"
 "vusenn"

In [127]:
get(dict_tables, "MinShareProduction", nothing)

,id,r,t,f,y,val
,Int64,String,String,String,String,Float64
1,1,costa_rica,supply_fuel_oil_from_fuel_electricity,fuel_oil,1000,0.0
2,2,costa_rica,supply_fuel_oil_from_fuel_electricity,fuel_oil,1001,0.15
3,3,costa_rica,supply_fuel_oil_from_fuel_electricity,fuel_oil,1002,0.3


In [130]:
t1 = get(dict_tables, "vproductionbytechnologyannual", nothing)
#filter(x -> (x[:t] == "pp_oil"), t1)
filter(x -> (x[:y] == "1002"), t1)


,r,t,f,y,val
,String,String,String,String,Float64
1,costa_rica,supply_fuel_crude,fuel_crude,1002,3.15576e11
2,costa_rica,pp_hydropower,fuel_electricity,1002,13.7627
3,costa_rica,pp_solar,fuel_electricity,1002,1.21603
4,costa_rica,st_compressed_air,fuel_electricity,1002,14.9759
5,costa_rica,st_batteries,fuel_electricity,1002,14.9787
6,costa_rica,supply_fuel_oil_from_fuel_electricity,fuel_oil,1002,22.5
7,costa_rica,st_flywheels,fuel_electricity,1002,14.9787
8,costa_rica,supply_fuel_solar,fuel_solar,1002,4.8255
9,costa_rica,supply_fuel_oil,fuel_oil,1002,52.5


In [139]:
(24*365*0.75*0.085)*1.298

724.8681

In [109]:
tab = filter(x -> ((x[:y] == "1002") & (x[:f] == "fuel_electricity")), t1)
sum(tab[:, :val]) - 21.7722

37.37555310931303

In [67]:
t2 = get(dict_tables, "InputActivityRatio", nothing)
filter(x -> ((x[:y] == "1000") & (x[:t] == "pp_oil")), t2)
166.262 - 36.947*2.5
37.2041*2.5

93.01024999999998

In [105]:
t3 = get(dict_tables, "SpecifiedAnnualDemand", nothing)
#filter(x -> ((x[:y] == "1001") & (x[:f] == "fuel_oil")), t3)
filter(x -> ((x[:y] == "1002")), t3)


,id,r,f,y,val
,Int64,String,String,String,Float64
1,3,costa_rica,fuel_electricity,1002,36.287
2,6,costa_rica,fuel_oil,1002,72.5739


In [88]:
i = 37
print("table_name: $(tabs_populated[i])")
df_cur = get(dict_tables, tabs_populated[i], nothing);
df_cur

table_name: CapacityFactor

,id,r,t,l,y,val
,Int64,String,String,String,String,Float64
1,1,costa_rica,pp_geothermal,fallwd0,1000,0.72
2,2,costa_rica,pp_geothermal,fallwd0,1001,0.72
3,3,costa_rica,pp_geothermal,fallwd0,1002,0.72
4,4,costa_rica,pp_geothermal,fallwd0,1003,0.72
5,5,costa_rica,pp_geothermal,fallwd0,1004,0.72
6,6,costa_rica,pp_geothermal,fallwd0,1005,0.72
7,7,costa_rica,pp_geothermal,fallwd0,1006,0.72
8,8,costa_rica,pp_geothermal,fallwd0,1007,0.72
9,9,costa_rica,pp_geothermal,fallwd0,1008,0.72


In [8]:
#DBInterface.execute(db, "select * from $(table_name)") |> DataFrame
#sort(collect(keys(dict_tables)))
#get(dict_tables, "CapacityFactor", nothing)

In [22]:
DBInterface.execute(db, "select * from CapitalCostStorage where (id in (1, 2, 3)) and (y in ('1005'))") |> DataFrame

,id,r,s,y,val
,Int64?,String?,String?,String?,Float64?


In [91]:
fp_test = normpath(joinpath(pathof(NemoMod), "..", "..", "test", "storage_test.sqlite"))


"/Users/jsyme/.julia/packages/NemoMod/vg6Tx/test/storage_test.sqlite"

In [115]:

model = Model(Gurobi.Optimizer)
#set_optimizer_attribute(model, "Solver", "cplex")
@time NemoMod.calculatescenario(
    fp_sqlite_nemomod_db_tmp; 
    jumpmodel = model,
    numprocs = 1
)

2022-31-Oct 06:35:20.388 Started modeling scenario.
2022-31-Oct 06:35:20.389 Validated run-time arguments.
2022-31-Oct 06:35:20.392 Connected to scenario database. Path = /Users/jsyme/Documents/Projects/git_jbus/lac_decarbonization/tmp/nemomod_intermediate_database.sqlite.
2022-31-Oct 06:35:20.399 Dropped pre-existing result tables from database.
2022-31-Oct 06:35:20.399 Verified that transmission modeling is not enabled.
2022-31-Oct 06:35:20.482 Created parameter views and indices.
2022-31-Oct 06:35:20.487 Created temporary tables.
2022-31-Oct 06:35:22.241 Executed core database queries.
2022-31-Oct 06:35:22.245 Defined dimensions.
2022-31-Oct 06:35:22.264 Defined demand variables.
2022-31-Oct 06:35:22.277 Defined storage variables.
2022-31-Oct 06:35:22.279 Defined capacity variables.
2022-31-Oct 06:35:25.271 Defined activity variables.
2022-31-Oct 06:35:25.275 Defined costing variables.
2022-31-Oct 06:35:25.276 Defined reserve margin variables.
2022-31-Oct 06:35:25.302 Defined emissi

INFEASIBLE_OR_UNBOUNDED::TerminationStatusCode = 6

In [114]:
typeof(model)

Model

In [106]:
MOI.compute_conflict!(model.moi_backend.optimizer.model)
MOI.get(model.moi_backend, Gurobi.ConstraintConflictStatus(), con1.index)



IIS runtime: 0.02 seconds (0.00 work units)


LoadError: Gurobi Error 10015: Cannot compute IIS on a feasible model